## Media Literacy: Using Fastai to teach K-12 students about Identity, privilage, and the importance of diversity

In [ ]:
# With the changing

In [ ]:
# Media literacy is defined as 


# Installing libraries

In [2]:
#hide
#hide_output
!apt-get install python3-pydot graphviz




E: Unable to locate package python3-pydot
E: Unable to locate package graphviz


In [3]:
#hide
#hide_output

!pip install fastai2
!pip install graphviz 
!pip install matplotlib
!pip install nbdev>=0.2.12 
!pip install pandas 
!pip install scikit_learn
!pip install azure-cognitiveservices-search-imagesearch
!pip install utils
!pip install Pillow
!pip install webp
!pip install ipywidgets
!pip install Voila>=2.4.1
!pip install voila-debug
!pip install voila-gridstack

In [6]:
#hide
!jupyter serverextension enable voila --sys-prefix


Enabling: voila
- Writing config: /opt/conda/envs/fastai/etc/jupyter
    - Validating...
      voila 0.1.21 OK


In [9]:
#hide
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [10]:
#hide_output
!git clone https://github.com/fastai/course-v4 'drive/My Drive/course-v4'


fatal: destination path 'drive/My Drive/course-v4' already exists and is not an empty directory.


In [11]:
#hide
from fastai2.vision.all import *
from fastai2 import *
from utils import *
from fastai2.vision.widgets import *

In [ ]:
#hide
from nbdev.showdoc import *
from ipywidgets import widgets
from pandas.api.types import CategoricalDtype

# Collecting data

In [ ]:
#hide
from azure.cognitiveservices.search.imagesearch import ImageSearchClient as api
from msrest.authentication import CognitiveServicesCredentials as auth

In [ ]:
#hide


In [ ]:
#hide
def search_images_bing(key, term, min_sz=128):
    client = api('https://api.cognitive.microsoft.com', auth(key))
    return L(client.images.search(query=term, count=150, min_height=min_sz, min_width=min_sz).value)

In [ ]:
#hide
results = search_images_bing(key, 'girls kids')
ims = results.attrgot('content_url')
len(ims)

In [ ]:
#hide
ims[0]


In [ ]:
#hide
kids= 'girl', 'boy'


In [ ]:
#hide
path = Path('media')
if not path.exists():
    path.mkdir()
    for o in kids:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} kids')
        download_images(dest, urls=results.attrgot('content_url'))

In [ ]:
#hide
fns = get_image_files(path)
fns

In [ ]:
#hide
from PIL import Image

In [ ]:
#hide_output
failed = verify_images(fns)
failed

In [ ]:
#hide
failed.map(Path.unlink);


# Showing the data and the labels

In [ ]:
#hide
media_data = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.3, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
#hide
dls= media_data.dataloaders(path)


In [ ]:
#hide_output
dls.valid.show_batch(max_n=4, nrows=1)


# Resizing the data

In [ ]:
#hide_output
media_data = media_data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = media_data.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

# Training 

In [ ]:
#hide_output
learn= cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)


In [ ]:
#hide_output
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
#hide_output
interp.plot_top_losses(5, nrows=1)


In [ ]:
#hide
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
for idx in cleaner.delete(): cleaner.fns[idx].unlink()


In [ ]:
#hide
learn.export()


# Prediction

In [ ]:
#hide_output
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
#hide
learn_inf = load_learner(path/'export.pkl', cpu=True)


In [ ]:
#hide_output
learn_inf.dls.vocab


In [ ]:
#hide
from IPython.display import display


In [ ]:
lbl_pred = widgets.Label()
out_pl = widgets.Output()


In [ ]:
btn_run = widgets.Button(description='Classify')


In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
btn_upload = widgets.FileUpload()


In [ ]:
display(VBox([widgets.Label('Am I represented?'), 
      btn_upload, btn_run, out_pl, lbl_pred]))

In [15]:
# Fixing the model 